<a href="https://colab.research.google.com/github/bohdanhlovatskyi/OhISee/blob/master/Camera_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import random as rng
from glob import glob

In [46]:

CHECKBOARD_SIZE = (10,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # precision of accurate pixels
winSize = (5,5) # area of check (5*2 + 1) - 11x11
zeroZone = (-1,-1) # no zeroZone

# creating zeo coordinates (0,0,0)
objp = np.zeros((CHECKBOARD_SIZE[0] * CHECKBOARD_SIZE[1], 3), np.float32)

# setting each possible coordinates 
"""
[0. 0. 0.] [0. 1. 0.]
[1. 0. 0.] [1. 1. 0.]  
[2. 0. 0.] [2. 1. 0.]
and so on
"""
objp[:,:2] = np.mgrid[0:CHECKBOARD_SIZE[0],0:CHECKBOARD_SIZE[1]].T.reshape(-1,2)

img_points = [] # 2d points
obj_points = [] # 3d points

In [47]:
images = glob("./*.jpg")
print(images)
for i in images:
  img_gray = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
  retval, corners = cv2.findChessboardCorners(img_gray, CHECKBOARD_SIZE)
  if not retval:
    raise Exception("Cannot find corners")

  # draw circles
  radius = 4
  copy = np.copy(img_gray)
  for i in range(corners.shape[0]):
    cv2.circle(copy, (int(corners[i,0,0]), 
                    int(corners[i,0,1])), 
              radius, (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)), cv2.FILLED)
  # cv2_imshow(copy)

  precise_corners = cv2.cornerSubPix(img_gray, corners, winSize, zeroZone, criteria)
  obj_points.append(objp)
  img_points.append(precise_corners)

print(img_points)
print(obj_points)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_gray.shape[::-1], None, None)
print(f"Camera matrix: \n{mtx}")

['./2022-04-12 15.40.01.jpg', './2022-04-12 15.39.39.jpg', './2022-04-12 15.39.43.jpg']
[array([[[265.73737, 479.62088]],

       [[299.35583, 479.44485]],

       [[333.49826, 479.23917]],

       [[368.28345, 478.76434]],

       [[403.57645, 478.4231 ]],

       [[439.57428, 477.88116]],

       [[476.39987, 477.47543]],

       [[513.9314 , 476.88602]],

       [[552.3414 , 476.33978]],

       [[591.606  , 475.58463]],

       [[262.29425, 514.9678 ]],

       [[295.96222, 515.1471 ]],

       [[330.3312 , 515.1688 ]],

       [[365.22275, 515.1872 ]],

       [[400.68033, 514.9522 ]],

       [[436.85062, 515.0252 ]],

       [[473.71454, 514.8152 ]],

       [[511.45157, 514.64813]],

       [[550.0254 , 514.4935 ]],

       [[589.5014 , 514.40045]],

       [[258.66528, 550.7288 ]],

       [[292.57895, 551.2253 ]],

       [[327.11456, 551.47845]],

       [[362.08942, 551.66705]],

       [[397.66718, 552.17615]],

       [[434.12497, 552.4387 ]],

       [[471.13046, 552.631